In [ ]:
import numpy as np
import cvxpy as cp
import cvxopt
import matplotlib.pyplot as plt
from AnsatzLacs import *
from PurificationUtil import *
from qiskit_algorithms.optimizers import SPSA
import sys

In [ ]:
# implementation of the slack variable method
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
total_size = sys_size
num_layers = 2

In [ ]:
# calculate the total variation distance
def total_variation_distance(rho, sigma):
    return 0.5 * np.sum(np.abs(rho - sigma))

In [ ]:
def get_trace(ansatz1, ansatz2, shots):
    '''
    Performs a descructive swap test (bell measurement) and expectation value calculation to produce the
    trace of the inner product of the state described by params1 and the state described by params2.
    To evaluate Tr[AB], ansatz1 should prepare state A and ansatz2 should prepare state B
    
    param shots: the number of shots executed on the quantum circuit over which to average
    return: floating point value for the trace
    '''
    probs1 = ansatz1.get_prob(shots)
    probs2 = ansatz2.get_prob(shots)
    return np.dot(probs1,probs2)

In [ ]:
rho_ansatz = QCBM(sys_size, 0, 2)
sigma_ansatz = QCBM(sys_size, 0, 2)

np.random.seed(1234)
rho_param = np.random.random(rho_ansatz.get_parameter_num())
sigma_param = np.random.random(sigma_ansatz.get_parameter_num())

rho_ansatz.set_params(rho_param)
sigma_ansatz.set_params(sigma_param)

rho = rho_ansatz.get_prob(None)
sigma = sigma_ansatz.get_prob(None)

expected = total_variation_distance(rho, sigma)
print("Exact value:", expected)

In [ ]:
rho_squared = get_trace(rho_ansatz, rho_ansatz, None)
sigma_squared = get_trace(sigma_ansatz, sigma_ansatz, None)
rho_sigma = get_trace(rho_ansatz, sigma_ansatz, None)

In [ ]:
# customized SPSA optimizer
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation * delta)
    minus = fun(params - perturbation * delta)
    grad = (plus - minus) / (2 * perturbation * delta)

    norm = np.linalg.norm(grad)
    if norm > 1:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .99, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Dual Optimization

In [ ]:
def cost_function_full_dual(params):
    """
    Returns the value of the cost function
    params: [parameters for 1, parameters for 2, lmbda, mu]
    """
    param1 = params[:num_parameters]
    param2 = params[num_parameters:2 * num_parameters]
    ansatz1.set_params(param1)
    ansatz2.set_params(param2)
    lmbda = (params[-2]) ** 2
    mu = (params[-1]) ** 2
    
    omega_squared = get_trace(ansatz1, ansatz1, num_shots)
    tau_squared = get_trace(ansatz2, ansatz2, num_shots)
    termA = lmbda**2 * omega_squared + rho_squared + sigma_squared + mu**2 * tau_squared
    
    omega_rho = get_trace(rho_ansatz, ansatz1, num_shots)
    omega_sigma = get_trace(sigma_ansatz, ansatz1, num_shots)
    omega_tau = get_trace(ansatz1, ansatz2, num_shots)
    termB = -lmbda * omega_rho + lmbda * omega_sigma - lmbda * mu * omega_tau
    
    rho_tau = get_trace(rho_ansatz, ansatz2, num_shots)
    sigma_tau = get_trace(sigma_ansatz, ansatz2, num_shots)
    termC = -rho_sigma + mu * rho_tau - mu * sigma_tau
    
    obj = lmbda
    
    penalty = termA + 2 * (termB + termC)
    
    cost = obj + c * penalty
    return cost, obj, penalty

def cost_function_dual(params):
    cost, obj, penalty = cost_function_full_dual(params)
    return cost

In [ ]:
print_every = 100
def callback_dual(x, iteration):
    global lr
    global lr_wait
    
    fx, obj, penal = cost_function_full_dual(x)

    dual_iterations.append(iteration)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)
    
    if (iteration % print_every==0):
        lr_wait += 1
        if lr_wait >= 3:
            m,_ = np.polyfit(dual_iterations[-300:], dual_loss[-300:], 1)
            if m > 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
                
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12
max_iter = 8000
lr_max = 1e-1
lr_min = 1e-3
perturbation = 1e-4
num_train = 10

In [ ]:
num_train = 3
record = 0

iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    c = 100
    
    np.random.seed(i)
    ansatz1 = create_ansatz('qcbm', sys_size, 0, num_layers)
    ansatz2 = create_ansatz('qcbm', sys_size, 0, num_layers)
    num_parameters = ansatz1.get_parameter_num()

    print("Expected value:", expected)
    init_params = [*ansatz1.params, *ansatz2.params, 1, 1]
    init_cost, init_obj, init_penal = cost_function_full_dual(init_params)
    
    dual_loss = [init_cost]
    dual_penals = [init_penal]
    dual_objectives = [init_obj]
    dual_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(cost_function_dual, init_params, get_gradient, max_iter, callback=callback_dual)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)
    iterations_dual_outer.append(dual_iterations)
    
    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"TVD_CSlack_dual_data"
        problem_name = "TVD"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Primal Optimization

In [ ]:
def cost_function_full_primal(params):
    """
    Returns the value of the cost function
    params: [parameters for 1, parameters for 2, lmbda, mu]
    """
    param1 = params[:num_parameters]
    param2 = params[num_parameters:2 * num_parameters]
    ansatz1.set_params(param1)
    ansatz2.set_params(param2)
    lmbda = (params[-2])**2
    mu = (params[-1])**2
    
    omega_squared = get_trace(ansatz1, ansatz1, num_shots)
    tau_squared = get_trace(ansatz2, ansatz2, num_shots)
    
    omega_rho = get_trace(rho_ansatz, ansatz1, num_shots)
    omega_sigma = get_trace(sigma_ansatz, ansatz1, num_shots)
    omega_tau = get_trace(ansatz1, ansatz2, num_shots)
    
    obj = lmbda * (omega_rho + -omega_sigma)
    
    penaltya = 2**sys_size + lmbda**2 * omega_squared + mu**2 * tau_squared
    penaltyb = lmbda * mu * omega_tau - lmbda - mu
    
    penalty = penaltya + 2 * penaltyb
    
    cost = obj - c * penalty
    return cost, obj, penalty

def cost_function_primal(params):
    cost, obj, penalty = cost_function_full_primal(params)
    return -cost

In [ ]:
print_every = 100
def callback_primal(x, iteration):
    global lr
    global lr_wait
    
    fx, obj, penal = cost_function_full_primal(x)

    primal_iterations.append(iteration)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)
    
    if (iteration % print_every==0):
        lr_wait += 1
        if lr_wait >= 3:
            m,_ = np.polyfit(primal_iterations[-300:], primal_loss[-300:], 1)
            if m < 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
                
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')

In [ ]:
num_shots = 1e12
max_iter = 8000
lr_max = 1e-1
lr_min = 1e-3
perturbation = 1e-4
num_train = 10

In [ ]:
num_train = 3
record = 0

iterations_primal_outer = []
costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    c = 10
    
    ansatz1 = create_ansatz('qcbm', sys_size, 0, num_layers)
    ansatz2 = create_ansatz('qcbm', sys_size, 0, num_layers)
    num_parameters = ansatz1.get_parameter_num()

    print("Expected value:", expected)
    init_params = [*ansatz1.params, *ansatz2.params, 1, 1]
    init_cost, init_obj, init_penal = cost_function_full_primal(init_params)
    
    primal_loss = [init_cost]
    primal_penals = [init_penal]
    primal_objectives = [init_obj]
    primal_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(cost_function_primal, init_params, get_gradient, max_iter, callback=callback_primal)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)
    iterations_primal_outer.append(primal_iterations)
    
    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"TVD_CSlack_primal_data"
        problem_name = "TVD"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 100
window_size_dual = 100

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
plt.ylim(-2, 5)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()